In [31]:
import ROOT

# PART 1

# look at slide 86

# define the observable
nobs = ROOT.RooRealVar("nobs", "n. of observed events", 0, 10)  # observable -> just the range
                       

In [32]:
# create a poisson model with signal strength
    
    # true, unknown n. of sig events 
mu = ROOT.RooRealVar("mu", "signal strength", 1, 0, 5)
nsig = ROOT.RooRealVar("nsig", "NOMINAL n. of signal events", 2.64 ) # CONSTANT (see Table 3 -> 2.64)

    # true, unknown n. of bkg events
nbkg = ROOT.RooRealVar("nbkg", "true, uknown n. of bkg events", 1, 0, 10) # PARAMETER 

    # true, unknown n. of total events
nexp = ROOT.RooFormulaVar("nexp", "@0 * @1 + @2", ROOT.RooArgList(mu, nsig, nbkg))    #  [mu, nsig, nbkg] -> in c++ is RooArgList(mu, nsig, nbkg)
  
   # poisson pdf

pois = ROOT.RooPoisson("pois", "pois", nobs, nexp)  

In [33]:
# create a guassian pdf to constraint background
# we need 3 variable: observable, mean and sigma

# we only have an estimate for "nbkg" with some uncertainty 
# see Table 3
# 0.25 ± 0.05


   # global observable (the estimate of nbkg)
   # Global Observable -> has a starting value but needs a range  
   # Global Observable we want it performs as a constant when we compute intervals 
   #                   we need it be float when we perform MC study  

nbkg_estimate = ROOT.RooRealVar("nbkg_estimate", "global observable", 0.25, 0, 10) 

   # sigma is 0.05
nbkg_err = ROOT.RooRealVar("sigma", "sigma", 0.05)  # CONSTANT (see Table 3 -> 0.05)

   # mean ? is already defined nbkg (true, unknown n. of bkg events)

gaus = ROOT.RooGaussian("gaus", "gaus", nbkg_estimate, nbkg, nbkg_err)


[#0] WARNING:InputArguments -- The parameter 'sigma' with range [-inf, inf] of the RooGaussian 'gaus' exceeds the safe range of (0, inf). Advise to limit its range.


In [34]:
# put together with RooProdPdf

model = ROOT.RooProdPdf("model", "1 channel numb. counting model with contraints", [pois, gaus])

In [35]:
# create the dataset with the observation of the experiment
# see Table 3

data = ROOT.RooDataSet("data", "", nobs)

nobs.setVal(5)
data.add(nobs)


In [36]:
# now we have the model, let's compute the p-value (significance)
# and test the hypothesis of a discovery 

# RooWorkspace

# ModelConfig

# run  ROOSTATS Calculator

In [37]:
# create a workspace and import the model and dataset

w = ROOT.RooWorkspace("w")

w.Import(model)   # in cpp is w.import(..) with 'i' lowercase
                  # with the model, automatically all cariables and formulas and pdfs are imported
w.Import(data)


False

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooProdPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooPoisson::pois
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nobs
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooFormulaVar::nexp
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mu
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nsig
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nbkg
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::gaus
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::nbkg_estimate
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::sigma
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset data


In [38]:
# check what is inside the workspace

w.Print()


RooWorkspace(w) w contents

variables
---------
(mu,nbkg,nbkg_estimate,nobs,nsig,sigma)

p.d.f.s
-------
RooGaussian::gaus[ x=nbkg_estimate mean=nbkg sigma=sigma ] = 1.38634e-49
RooProdPdf::model[ pois * gaus ] = 1.93805e-50
RooPoisson::pois[ x=nobs mean=nexp ] = 0.139796

functions
--------
RooFormulaVar::nexp[ actualVars=(mu,nsig,nbkg) formula="@0 * @1 + @2" ] = 3.64

datasets
--------
RooDataSet::data(nobs)



In [39]:
# REMIND... how to retrieve objects from the model?

w.var("nbkg_estimate").Print()
w.pdf("gaus").Print()
w.function("nexp").Print()
w.data("data").Print()

RooRealVar::nbkg_estimate = 0.25  L(0 - 10) 
RooGaussian::gaus[ x=nbkg_estimate mean=nbkg sigma=sigma ] = 1.38634e-49
RooFormulaVar::nexp[ actualVars=(mu,nsig,nbkg) formula="@0 * @1 + @2" ] = 3.64
RooDataSet::data[nobs] = 1 entries


In [40]:
# create the Model Config
# we need to clarify the role of the objects in our analysis

mc = ROOT.RooStats.ModelConfig("model_config")
mc.SetWorkspace(w)
mc.SetPdf(w.pdf("model"))
mc.SetParametersOfInterest(w.var("mu")) # the variable where interested in
mc.SetObservables(w.var("nobs"))

mc.SetGlobalObservables(w.var("nbkg_estimate"))  # the estimate of brackground constrained by the gaussian
w.var("nbkg_estimate").setConstant(True)         # this is a requirement for Global Observables
# The reason is that b0 needs to be treated as an auxiliary observable in case of frequentist statistics and varied when tossing pseudo-experiments.

# all other variables are nuisance parameters
mc.SetNuisanceParameters(w.var("nbkg"))

mc.Print()

# import into the workspace

w.Import(mc)


False


=== Using the following for model_config ===
Observables:             RooArgSet:: = (nobs)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (nbkg)
Global Observables:      RooArgSet:: = (nbkg_estimate)
PDF:                     RooProdPdf::model[ pois * gaus ] = 1.93805e-50



In [41]:
# write the Workspace to a file

w.writeToFile("opera_1channel.root")

False

```
Windows PowerShell
Copyright (C) Microsoft Corporation. Tutti i diritti riservati.

Installa la versione più recente di PowerShell per nuove funzionalità e miglioramenti. https://aka.ms/PSWindows

PS C:\WINDOWS\system32> wsl
gabrielesirri2@SGCSA-LZ00D:/mnt/c/WINDOWS/system32$ root
   ------------------------------------------------------------------
  | Welcome to ROOT 6.36.04                        https://root.cern |
  | (c) 1995-2025, The ROOT Team; conception: R. Brun, F. Rademakers |
  | Built for linuxx8664gcc on Aug 25 2025, 09:43:16                 |
  | From tags/v6-36-04@v6-36-04                                      |
  | With c++ (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0                   |
  | Try '.help'/'.?', '.demo', '.license', '.credits', '.quit'/'.q'  |
   ------------------------------------------------------------------

root [0] .L $ROOTSYS/
LICENSE
README/
bin/
cmake/
config/
etc/
fonts/
geom/
icons/
include/
js/
lib/
macros/
man/
tutorials/
ui5/
root [0] .L $ROOTSYS/tutorials/roo
roofit/
rootlogoff.C
rootlogon.C
root [0] .L $ROOTSYS/tutorials/roofit/
histfactory/
index.md
roofit/
roostats/
root [0] .L $ROOTSYS/tutorials/roofit/roo
roofit/
roostats/
root [0] .L $ROOTSYS/tutorials/roofit/roostats/Standard
StandardBayesianMCMCDemo.C
StandardBayesianMCMCDemo.py
StandardBayesianNumericalDemo.C
StandardBayesianNumericalDemo.py
StandardFeldmanCousinsDemo.C
StandardFeldmanCousinsDemo.py
StandardFrequentistDiscovery.C
StandardFrequentistDiscovery.py
StandardHistFactoryPlotsWithCategories.C
StandardHistFactoryPlotsWithCategories.py
StandardHypoTestDemo.C
StandardHypoTestDemo.py
StandardHypoTestInvDemo.C
StandardHypoTestInvDemo.py
StandardProfileInspectorDemo.C
StandardProfileInspectorDemo.py
StandardProfileLikelihoodDemo.C
StandardProfileLikelihoodDemo.py
StandardTestStatDistributionDemo.C
StandardTestStatDistributionDemo.py
root [0] .L $ROOTSYS/tutorials/roofit/roostats/StandardHypoTestDemo.
StandardHypoTestDemo.C
StandardHypoTestDemo.py
root [0] .L $ROOTSYS/tutorials/roofit/roostats/StandardHypoTestDemo.C
root [1] StandardHypoTestDemo(
void StandardHypoTestDemo(const char* infile = "", const char* workspaceName = "combined", const char* modelSBName = "ModelConfig", const char* modelBName = "", const char* dataName = "obsData", int calcType = 0, int testStatType = 3, int ntoys = 5000, bool useNC = false, const char* nuisPriorName = 0)
root [1] .q
gabrielesirri2@SGCSA-LZ00D:/mnt/c/WINDOWS/system32$ cp /mnt/c/
$Recycle.Bin/              PanoptoRecorder/           Programmi/                 _rotational_comments.txt
DesktopLinux/              PerfLogs/                  Recovery/                  hiberfil.sys
Documents and Settings/    Program Files/             System Volume Information/ inetpub/
DumpStack.log.tmp          Program Files (x86)/       Users/                     pagefile.sys
GPCeSIA/                   ProgramData/               Windows/                   swapfile.sys
gabrielesirri2@SGCSA-LZ00D:/mnt/c/WINDOWS/system32$ cp /mnt/c/Users/gabriele.sirri2/Desktop/opera_1channel.root .
cp: cannot create regular file './opera_1channel.root': Permission denied
gabrielesirri2@SGCSA-LZ00D:/mnt/c/WINDOWS/system32$ cd $HOME
gabrielesirri2@SGCSA-LZ00D:~$ cp /mnt/c/Users/gabriele.sirri2/Desktop/opera_1channel.root .
gabrielesirri2@SGCSA-LZ00D:~$ ls
Untitled-2.ipynb   opera_1channel.root  root
labsession2.ipynb  rooaddpdf.ipynb      root_v6.36.04.Linux-ubuntu24.04-x86_64-gcc13.3.tar.gz
gabrielesirri2@SGCSA-LZ00D:~$ root -l opera_1channel.root
root [0]
Attaching file opera_1channel.root as _file0...
(TFile *) 0x5780e2ff53a0
root [1] .ls
TFile**         opera_1channel.root
 TFile*         opera_1channel.root
  KEY: RooWorkspace     w;1     w
  KEY: TProcessID       ProcessID0;1    b61426b6-cc3f-11f0-ab24-bfd61eacbeef
root [2] w
(RooWorkspace *) 0x5780e3dd4770
root [3] w.Print()
ROOT_prompt_3:1:2: error: member reference type 'RooWorkspace *' is a pointer; did you mean to use '->'?
w.Print()
~^
 ->
root [4] w->Print()

RooWorkspace(w) w contents

variables
---------
(mu,nbkg,nbkg_estimate,nobs,nsig,sigma)

p.d.f.s
-------
RooGaussian::gaus[ x=nbkg_estimate mean=nbkg sigma=sigma ] = 1.38634e-49
RooProdPdf::model[ pois * gaus ] = 1.93805e-50
RooPoisson::pois[ x=nobs mean=nexp ] = 0.139796

functions
--------
RooFormulaVar::nexp[ actualVars=(mu,nsig,nbkg) formula="@0 * @1 + @2" ] = 3.64

datasets
--------
RooDataSet::data(nobs)

named sets
----------
model_config_GlobalObservables:(nbkg_estimate)
model_config_NuisParams:(nbkg)
model_config_Observables:(nobs)
model_config_POI:(mu)

generic objects
---------------
RooStats::ModelConfig::model_config

root [5] .L $ROOTSYS/
LICENSE
README/
bin/
cmake/
config/
etc/
fonts/
geom/
icons/
include/
js/
lib/
macros/
man/
tutorials/
ui5/
root [5] .L $ROOTSYS/tutorials/roo
roofit/
rootlogoff.C
rootlogon.C
root [5] .L $ROOTSYS/tutorials/roofit/roostats/StandardHypoTestDemo.C
root [6] // StandardHypoTestDemo("opera_1channel.root", "w", "model_config
root [7]  3 )
root [7]
root [7] StandardHypoTestDemo("opera_1channel.root", "w", "model_config", "", 2, 3, 0 /* ntoys*/, true /* useNC */)
ROOT_prompt_7:1:1: error: no matching function for call to 'StandardHypoTestDemo'
StandardHypoTestDemo("opera_1channel.root", "w", "model_config", "", 2, 3, 0 /* ntoys*/, true /* useNC */)
^~~~~~~~~~~~~~~~~~~~
/home/gabrielesirri2/root/tutorials/roofit/roostats/StandardHypoTestDemo.C:78:6: note: candidate function not viable: no known conversion from 'int' to 'const char *' for 5th argument
void StandardHypoTestDemo(const char *infile = "", const char *workspaceName = "combined",
     ^
root [8] StandardHypoTestDemo("opera_1channel.root", "w", "model_config", "", "data", 2, 3, 0 /* ntoys*/, true /* u
seNC */)
RooWorkspace(w) w contents

variables
---------
(mu,nbkg,nbkg_estimate,nobs,nsig,sigma)

p.d.f.s
-------
RooGaussian::gaus[ x=nbkg_estimate mean=nbkg sigma=sigma ] = 1.38634e-49
RooProdPdf::model[ pois * gaus ] = 1.93805e-50
RooPoisson::pois[ x=nobs mean=nexp ] = 0.139796

functions
--------
RooFormulaVar::nexp[ actualVars=(mu,nsig,nbkg) formula="@0 * @1 + @2" ] = 3.64

datasets
--------
RooDataSet::data(nobs)

named sets
----------
model_config_GlobalObservables:(nbkg_estimate)
model_config_NuisParams:(nbkg)
model_config_Observables:(nobs)
model_config_POI:(mu)

generic objects
---------------
RooStats::ModelConfig::model_config

Info in <StandardHypoTestInvDemo>: The background model  does not exist
Info in <StandardHypoTestInvDemo>: Copy it from ModelConfig model_config and set POI to zero
Info in <StandardHypoTestDemo>: Model model_config has no snapshot  - make one using model poi
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize - Find  best unconditional NLL on observed data
[#0] PROGRESS:Eval -- Best fitted POI value = 1.84289 +/- 0.894172
[#0] PROGRESS:Eval -- AsymptoticCalculator: Building Asimov data Set
[#1] INFO:InputArguments -- AsymptoticCalculator: Asimov data will be generated using fitted nuisance parameter values
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize Find  best conditional NLL on ASIMOV data set for given alt POI ( mu ) = 1

Results HypoTestAsymptotic_result:
 - Null p-value = 4.47597e-06
 - Significance = 4.44105
 - CL_b: 4.47597e-06
 - CL_s+b: 0.0654907
 - CL_s: 14631.6
Asymptotic results
 Expected p -value and significance at -2 sigma = 0.175976 significance 0.930811 sigma
 Expected p -value and significance at -1 sigma = 0.0267532 significance 1.93081 sigma
 Expected p -value and significance at 0 sigma = 0.00169039 significance 2.93081 sigma
 Expected p -value and significance at 1 sigma = 4.23299e-05 significance 3.93081 sigma
 Expected p -value and significance at 2 sigma = 4.09445e-07 significance 4.93081 sigma
root [9]
```




















































In [46]:
# create the class using data and model config
plc = ROOT.RooStats.ProfileLikelihoodCalculator (data, mc)

# set value of POI to mu_zero (this case is 0)
# one can also use mc.SetSnapshot(*mu)
mu.setVal(0);
plc.SetNullParameters(mu)
hypotest = plc.GetHypoTest()
alpha = hypotest.NullPValue()
significance = hypotest.Significance()

print(significance)
print(alpha)

4.440999168167042
4.4771070533140425e-06
[#1] INFO:InputArguments -- The deprecated RooFit::CloneData(1) option passed to createNLL() is ignored.
[#1] INFO:Minimization --  Including the following constraint terms in minimization: (gaus)
[#1] INFO:Minimization -- The following global observables have been defined and their values are taken from the model: (nbkg_estimate)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 328.093 μs
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_data) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#0] PROGRESS:Minimization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit2 /  with strategy 1
[#1] INFO:Minimization -- 
  RooFitRes

In [ ]:

# set the confidence level
plc.SetConfidenceLevel(0.683);

# compute the interval of the parameter mu
interval = plc.GetInterval();

# print the interval (get a pointer of mu from M.C.)
poi = mc.GetParametersOfInterest().first()
lowerLimit = interval.LowerLimit(poi)
upperLimit = interval.UpperLimit(poi)

# plot the interval
plot = ROOT.RooStat.LikelihoodIntervalPlot(interval)
plot.Draw()